# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import scipy.stats as st
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key



In [4]:
csv_file="mlb_games.csv"

mlb_games_df=pd.read_csv(csv_file,sep='[,]',engine="python",index_col=7)



mlb_games_df.head()



,"""away_team",away_team_score,date,game_id,home_team,home_team_score,note,season,"year"""
NaN,"""chw",4,"""""Thu","Mar 4""""",300304103.0,laa,4,Preseason,"2010"""
NaN,"""lad",8,"""""Fri","Mar 5""""",300305104.0,chw,3,Preseason,"2010"""
NaN,"""chw",15,"""""Sat","Mar 6""""",300306116.0,chc,3,Preseason,"2010"""
NaN,"""chw",8,"""""Sat","Mar 6""""",300306119.0,lad,4,Preseason,"2010"""
NaN,"""sea",5,"""""Mon","Mar 8""""",300308104.0,chw,4,Preseason,"2010"""


In [5]:
mlb_games_df["time"]=mlb_games_df[["date","game_id",'year"']].agg('-'.join, axis=1)
mlb_games_df.head()               

,"""away_team",away_team_score,date,game_id,home_team,home_team_score,note,season,"year""",time
NaN,"""chw",4,"""""Thu","Mar 4""""",300304103.0,laa,4,Preseason,"2010""","""""Thu- Mar 4""""-2010"""
NaN,"""lad",8,"""""Fri","Mar 5""""",300305104.0,chw,3,Preseason,"2010""","""""Fri- Mar 5""""-2010"""
NaN,"""chw",15,"""""Sat","Mar 6""""",300306116.0,chc,3,Preseason,"2010""","""""Sat- Mar 6""""-2010"""
NaN,"""chw",8,"""""Sat","Mar 6""""",300306119.0,lad,4,Preseason,"2010""","""""Sat- Mar 6""""-2010"""
NaN,"""sea",5,"""""Mon","Mar 8""""",300308104.0,chw,4,Preseason,"2010""","""""Mon- Mar 8""""-2010"""


In [6]:
mlb_games_df1=mlb_games_df.loc[:,mlb_games_df.dtypes==object].apply(lambda s:s.str.replace('"', ""))
mlb_games_df1.head()

,"""away_team",away_team_score,date,game_id,home_team_score,note,season,"year""",time
NaN,chw,4,Thu,Mar 4,laa,4,Preseason,2010,Thu- Mar 4-2010
NaN,lad,8,Fri,Mar 5,chw,3,Preseason,2010,Fri- Mar 5-2010
NaN,chw,15,Sat,Mar 6,chc,3,Preseason,2010,Sat- Mar 6-2010
NaN,chw,8,Sat,Mar 6,lad,4,Preseason,2010,Sat- Mar 6-2010
NaN,sea,5,Mon,Mar 8,chw,4,Preseason,2010,Mon- Mar 8-2010


In [7]:
mlb_games_df1.insert(2,"id",mlb_games_df["home_team"])
mlb_games_df1.head()

,"""away_team",away_team_score,id,date,game_id,home_team_score,note,season,"year""",time
NaN,chw,4,300304103.0,Thu,Mar 4,laa,4,Preseason,2010,Thu- Mar 4-2010
NaN,lad,8,300305104.0,Fri,Mar 5,chw,3,Preseason,2010,Fri- Mar 5-2010
NaN,chw,15,300306116.0,Sat,Mar 6,chc,3,Preseason,2010,Sat- Mar 6-2010
NaN,chw,8,300306119.0,Sat,Mar 6,lad,4,Preseason,2010,Sat- Mar 6-2010
NaN,sea,5,300308104.0,Mon,Mar 8,chw,4,Preseason,2010,Mon- Mar 8-2010


In [8]:
mlb_games_df2=pd.DataFrame(mlb_games_df1)
mlb_games_df2.head()



,"""away_team",away_team_score,id,date,game_id,home_team_score,note,season,"year""",time
NaN,chw,4,300304103.0,Thu,Mar 4,laa,4,Preseason,2010,Thu- Mar 4-2010
NaN,lad,8,300305104.0,Fri,Mar 5,chw,3,Preseason,2010,Fri- Mar 5-2010
NaN,chw,15,300306116.0,Sat,Mar 6,chc,3,Preseason,2010,Sat- Mar 6-2010
NaN,chw,8,300306119.0,Sat,Mar 6,lad,4,Preseason,2010,Sat- Mar 6-2010
NaN,sea,5,300308104.0,Mon,Mar 8,chw,4,Preseason,2010,Mon- Mar 8-2010


In [9]:

mlb_games_df2.columns=["away_team","away_team_score","gameid","day","date","home_team","home_team_score","season","year","time"]

mlb_games_df2.head()



,away_team,away_team_score,gameid,day,date,home_team,home_team_score,season,year,time
NaN,chw,4,300304103.0,Thu,Mar 4,laa,4,Preseason,2010,Thu- Mar 4-2010
NaN,lad,8,300305104.0,Fri,Mar 5,chw,3,Preseason,2010,Fri- Mar 5-2010
NaN,chw,15,300306116.0,Sat,Mar 6,chc,3,Preseason,2010,Sat- Mar 6-2010
NaN,chw,8,300306119.0,Sat,Mar 6,lad,4,Preseason,2010,Sat- Mar 6-2010
NaN,sea,5,300308104.0,Mon,Mar 8,chw,4,Preseason,2010,Mon- Mar 8-2010


In [10]:
del mlb_games_df2["day"]
del mlb_games_df2["date"]
del mlb_games_df2["year"]
mlb_games_df2.head()

,away_team,away_team_score,gameid,home_team,home_team_score,season,time
NaN,chw,4,300304103.0,laa,4,Preseason,Thu- Mar 4-2010
NaN,lad,8,300305104.0,chw,3,Preseason,Fri- Mar 5-2010
NaN,chw,15,300306116.0,chc,3,Preseason,Sat- Mar 6-2010
NaN,chw,8,300306119.0,lad,4,Preseason,Sat- Mar 6-2010
NaN,sea,5,300308104.0,chw,4,Preseason,Mon- Mar 8-2010


In [11]:
csv_file="mlb_teams.csv"

mlb_teams_df=pd.read_csv(csv_file,sep='[,]',engine="python")



mlb_teams_df.head()

,"""league",division,team,prefix_1,"prefix_2"""
0,"""mlb",AL Central,Chicago White Sox,chw,"chicago-white-sox"""
1,"""mlb",AL Central,Cleveland Indians,cle,"cleveland-indians"""
2,"""mlb",AL Central,Detroit Tigers,det,"detroit-tigers"""
3,"""mlb",AL Central,Kansas City Royals,kc,"kansas-city-royals"""
4,"""mlb",AL Central,Minnesota Twins,min,"minnesota-twins"""


In [12]:
mlb_teams_df["city"]=mlb_teams_df["team"].str.split(" ").str[0]
mlb_teams_df["city"].replace({"New":"New York","St.":"St. Louis","Los":"Los Angeles","San":"San Diego"},inplace=True)
#mlb_teams_df2=mlb_teams_df.loc([:,"prefix_1"=="sf"].map({"San Diego":"San Francisco"}))

mlb_teams_df.at[29,"city"]="San Francisco"
mlb_teams_df

,"""league",division,team,prefix_1,"prefix_2""",city
0,"""mlb",AL Central,Chicago White Sox,chw,"chicago-white-sox""",Chicago
1,"""mlb",AL Central,Cleveland Indians,cle,"cleveland-indians""",Cleveland
2,"""mlb",AL Central,Detroit Tigers,det,"detroit-tigers""",Detroit
3,"""mlb",AL Central,Kansas City Royals,kc,"kansas-city-royals""",Kansas
4,"""mlb",AL Central,Minnesota Twins,min,"minnesota-twins""",Minnesota
5,"""mlb",AL East,Baltimore Orioles,bal,"baltimore-orioles""",Baltimore
6,"""mlb",AL East,Boston Red Sox,bos,"boston-red-sox""",Boston
7,"""mlb",AL East,New York Yankees,nyy,"new-york-yankees""",New York
8,"""mlb",AL East,Tampa Bay Rays,tb,"tampa-bay-rays""",Tampa
9,"""mlb",AL East,Toronto Blue Jays,tor,"toronto-blue-jays""",Toronto


In [13]:

mlb_teams_df1=mlb_teams_df[["prefix_1","city"]].copy()
mlb_teams_df1.head()

,prefix_1,city
0,chw,Chicago
1,cle,Cleveland
2,det,Detroit
3,kc,Kansas
4,min,Minnesota


In [14]:
mlb_teams_df1.columns=("home_team","city")
mlb_teams_df1

,home_team,city
0,chw,Chicago
1,cle,Cleveland
2,det,Detroit
3,kc,Kansas
4,min,Minnesota
5,bal,Baltimore
6,bos,Boston
7,nyy,New York
8,tb,Tampa
9,tor,Toronto


In [15]:
df=pd.merge(mlb_games_df2,mlb_teams_df,left_on="home_team",right_on="prefix_1",how="left")
df.head()

,away_team,away_team_score,gameid,home_team,home_team_score,season,time,"""league",division,team,prefix_1,"prefix_2""",city
0,chw,4,300304103.0,laa,4,Preseason,Thu- Mar 4-2010,"""mlb",AL West,Los Angeles Angels,laa,"los-angeles-angels""",Los Angeles
1,lad,8,300305104.0,chw,3,Preseason,Fri- Mar 5-2010,"""mlb",AL Central,Chicago White Sox,chw,"chicago-white-sox""",Chicago
2,chw,15,300306116.0,chc,3,Preseason,Sat- Mar 6-2010,"""mlb",NL Central,Chicago Cubs,chc,"chicago-cubs""",Chicago
3,chw,8,300306119.0,lad,4,Preseason,Sat- Mar 6-2010,"""mlb",NL West,Los Angeles Dodgers,lad,"los-angeles-dodgers""",Los Angeles
4,sea,5,300308104.0,chw,4,Preseason,Mon- Mar 8-2010,"""mlb",AL Central,Chicago White Sox,chw,"chicago-white-sox""",Chicago


In [16]:
df.tail()

,away_team,away_team_score,gameid,home_team,home_team_score,season,time,"""league",division,team,prefix_1,"prefix_2""",city
31852,sf,7,360322129.0,ari,7 F/10,Preseason,Tue- Mar 22-2016,"""mlb",NL West,Arizona Diamondbacks,ari,"arizona-diamondbacks""",Arizona
31853,vsSacramento,4,360330126.0,sf,8,Preseason,Wed- Mar 30-2016,"""mlb",NL West,San Francisco Giants,sf,"san-francisco-giants""",San Francisco
31854,vsPuerto Rico,5,370308126.0,sf,6,Preseason,Wed- Mar 8-2017,"""mlb",NL West,San Francisco Giants,sf,"san-francisco-giants""",San Francisco
31855,sd,4 F/10,380307126.0,sf,4,Preseason,Wed- Mar 7-2018,"""mlb",NL West,San Francisco Giants,sf,"san-francisco-giants""",San Francisco
31856,@Sacramento,6,380324152.0,sf,5,Preseason,Sat- Mar 24-2018,"""mlb",NL West,San Francisco Giants,sf,"san-francisco-giants""",San Francisco
